### Welcome to the third class on E-ID in this 3-class series
Before we start, make sure you have everything installed by following one of the outlined methods in the ReadMe file. 

### Purpose of this exercise
This jupyter notebook explains the following:
1. Zero-Knowledge proofs and specifically range proofs using BBS
2. How to use them using BulletProof++
3. How to use them using LegoGroth16
4. All examples here will use the **docknetwork crypto library**

In [6]:
const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        profession: { type: 'string' },
        age: { type: 'integer', minimum: 0, maximum: 120},
    }
}
const E_ID_DATA = {
    name: "Jack Sparrow",
    age: 61,
    profession: "IT Manager"
}

In [2]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

## Issuer

In [5]:
// Generating a keypair once signature parameters are created.
import { BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES } from '@docknetwork/crypto-wasm-ts';

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));

const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

In [8]:
// Issing a credential based on the E-ID data based on the specified schema
import { CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'

const schema = CredentialSchema.essential();
schema.properties[SUBJECT_STR] = E_ID_SCHEMA

const builder = new BBSCredentialBuilder();
builder.schema = new CredentialSchema(schema);
builder.subject = E_ID_DATA

const credential = builder.sign(secretKey);

const credentialToSendToHolder = credential.toJSON()

## Credential Holder

In [9]:
// This section establishes a range proof capability based on the LegoGroth16 algorithm
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

In [10]:
import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
builder.enforceBounds(0, 'credentialSubject.age', 18, 64, 'random', snarkProvingKey);

const presentation = builder.finalize();

### sending data to Verifier
data: 
1. selected data to be revealed
2. Schema of the credential
3. Proof of signature
4. 

All these info and more are stored in the "presentation object". Note that this object is a completely serializable JSON object.

## Verifier

In [12]:
const receivedObjectOnVerifierSide = presentation.toJSON()
receivedObjectOnVerifierSide  // Assume this was serialized and sent across an API to the verifier.
presentation.spec.credentials[0]

{
  version: '0.7.0',
  schema: {
    details: '{"jsonSchema":{"$schema":"http://json-schema.org/draft-07/schema#","definitions":{"encryptableCompString":{"type":"string"},"encryptableString":{"type":"string"}},"properties":{"credentialSubject":{"properties":{"age":{"maximum":120,"minimum":0,"type":"integer"},"name":{"type":"string"},"profession":{"type":"string"}},"type":"object"},"proof":{"properties":{"type":{"type":"string"}},"type":"object"}},"type":"object"},"parsingOptions":{"defaultDecimalPlaces":0,"defaultMinimumDate":-17592186044415,"defaultMinimumInteger":-4294967295,"useDefaults":false}}',
    id: 'data:application/json;charset=utf-8,',
    type: 'JsonSchemaValidator2018',
    version: '0.5.0'
  },
  revealedAttributes: {
    credentialSubject: { name: 'Jack Sparrow', profession: 'IT Manager' }
  },
  bounds: { credentialSubject: { age: [Array] } },
  sigType: 'Bls12381BBSSignatureDock2023'
}


In [13]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

const predicateParams = new Map([['random', snarkVerifyingKey]]);

const recreatedPres = Presentation.fromJSON(receivedObjectOnVerifierSide)
recreatedPres.verify([publicKey], undefined, predicateParams, undefined, undefined, undefined)

{ verified: true, error: undefined }


In [18]:
presentation.spec.credentials[0].bounds.credentialSubject

{
  age: [ { min: 18, max: 64, paramId: 'random', protocol: 'LegoGroth16' } ]
}
